In [1]:
# required installs:
# numpy, pandas, seaborn, selenium, chromedriver-py, requests, beautifulsoup4

# all installs can be installed with pip 
#  pip install [package]

# if chromedriver-py doesn't work, dl from here (https://sites.google.com/a/chromium.org/chromedriver/home)

import numpy as np
import pandas as pd
# import seaborn as sns

import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from chromedriver_py import binary_path # this will get you the path variable

import requests
from bs4 import BeautifulSoup

import time
# import lxml
# from lxml import etree

In [28]:
# use just beautifulsoup to open webpage
url = 'https://na.op.gg/summoner/userName=FriedenInGeduld'
response = requests.get(url)
page = response.text
soup = BeautifulSoup(page, 'lxml')
print(soup.prettify())

FeatureNotFound: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?

In [14]:
# use selenium to open webpage


driver = webdriver.Chrome(executable_path=binary_path)
driver.get("https://www.amazon.com/gp/css/order-history?ref_=nav_AccountFlyout_orders")
try:
    #driver.manage().timeouts().implicitlyWait(5, TimeUnit.SECONDS);

    # !!! CLEAR THESE BEFORE PUSHING !!!
    email = driver.find_element_by_name("email")
    email.clear()
    email.send_keys("") # left out for safety
    email.send_keys(Keys.RETURN)

    pwd = driver.find_element_by_name("password")
    pwd.clear()
    pwd.send_keys("") # left out for safety
    pwd.send_keys(Keys.RETURN)
    
    #first_order = driver.find_element_by_class_name('a-box-group a-spacing-base order')
    orders_box = driver.find_element_by_id('ordersContainer')
    orders = orders_box.find_elements_by_class_name('a-box-group')
    for i in orders:
        
    print(len(orders))
    
    time.sleep(10)
finally:
    driver.quit()
driver.quit()

10
